# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4973754598408747105
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357564416
locality {
  bus_id: 1
}
incarnation: 18026184180646276140
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


In [5]:
type(english_sentences)

list

In [6]:
english_sentences[0]

'new jersey is sometimes quiet during autumn , and it is snowy in april .'

In [7]:
french_sentences[0]

"new jersey est parfois calme pendant l' automne , et il est neigeux en avril ."

In [8]:
english_sentences[1]

'the united states is usually chilly during july , and it is usually freezing in november .'

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [9]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [10]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))



1823250 English words.
227 unique English words.


In [11]:
sum(english_words_counter.values())

1823250

In [12]:
english_words_counter.most_common(10)

[('is', 205858),
 (',', 140897),
 ('.', 129039),
 ('in', 75525),
 ('it', 75137),
 ('during', 74933),
 ('the', 67628),
 ('but', 63987),
 ('and', 59850),
 ('sometimes', 37746)]

In [13]:
len(list(french_words_counter.keys()))

355

In [14]:
len(list(english_words_counter.keys()))

227

In [15]:
list(zip(*english_words_counter.most_common(10)))

[('is', ',', '.', 'in', 'it', 'during', 'the', 'but', 'and', 'sometimes'),
 (205858, 140897, 129039, 75525, 75137, 74933, 67628, 63987, 59850, 37746)]

In [16]:
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [17]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    token = Tokenizer()
    token.fit_on_texts(x)
    return token.texts_to_sequences(x), token
tests.test_tokenize(tokenize)

# # Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [18]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(i) for i in x])
    
    return pad_sequences(x, maxlen= length, padding='post')
tests.test_pad(pad)
  
# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [19]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)
    print(preprocess_y.shape)
    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    print('after',preprocess_y.shape)
    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

(137861, 21)
after (137861, 21, 1)
Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [20]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [21]:
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional

def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = .001
    input_seq = Input(input_shape[1:])
    gru = GRU(512, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(gru)
    model=Model(input_seq, Activation('tanh')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)



# Reshaping the input to work with a basic RNN

tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
# tmp_x = tmp_x.reshape(-1, preproc_french_sentences.shape[-2])
print(tmp_x.shape)
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=8, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(137861, 21)
(137861, 21, 1)
Train on 110288 samples, validate on 27573 samples
Epoch 1/8
110288/110288 [==============================] - 40s 366us/step - loss: 1.3576 - acc: 0.6256 - val_loss: nan - val_acc: 0.6725
Epoch 2/8
110288/110288 [==============================] - 38s 348us/step - loss: 0.9514 - acc: 0.6850 - val_loss: nan - val_acc: 0.7040
Epoch 3/8
110288/110288 [==============================] - 38s 348us/step - loss: 0.7858 - acc: 0.7392 - val_loss: nan - val_acc: 0.7630
Epoch 4/8
110288/110288 [==============================] - 38s 348us/step - loss: 0.6667 - acc: 0.7835 - val_loss: nan - val_acc: 0.7917
Epoch 5/8
110288/110288 [==============================] - 38s 348us/step - loss: 0.6061 - acc: 0.8021 - val_loss: nan - val_acc: 0.7955
Epoch 6/8
110288/110288 [==============================] - 38s 348us/step - loss: 0.5510 - acc: 0.8200 - val_loss: nan - val_acc: 0.8416
Epoch 7/8
110288/110288 [==============================] - 38s 348us/step - loss: 0.5257 - acc: 0.

In [22]:
simple_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_2 (GRU)                  (None, 21, 512)           789504    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 21, 344)           176472    
_________________________________________________________________
activation_2 (Activation)    (None, 21, 344)           0         
Total params: 965,976
Trainable params: 965,976
Non-trainable params: 0
_________________________________________________________________


In [23]:
preproc_french_sentences.shape[-2]

21

In [24]:
preproc_french_sentences.shape

(137861, 21, 1)

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [25]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    print(input_shape, 'input shape')
    embedding = Embedding(input_dim = english_vocab_size, output_dim = 128)(input_seq)
    gru = GRU(512, return_sequences= True)(embedding)
    logits= TimeDistributed(Dense( french_vocab_size, activation='softmax'))(gru)
    model = Model(input_seq, Activation('tanh')(logits))
      
    model.compile(loss= sparse_categorical_crossentropy,
                 optimizer=Adam(learning_rate),
                 metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)
tmp_x2 = tmp_x.reshape(-1, preproc_french_sentences.shape[-2])
print(preproc_french_sentences.shape[-2])
print(tmp_x2.shape)
# TODO: Train the neural network

embed_model = embed_model(tmp_x2.shape, 
                          max_french_sequence_length,
                          english_vocab_size, french_vocab_size)


(137861, 21) input shape
(137861, 21)
21
(137861, 21)
(137861, 21) input shape


In [26]:
embed_model.fit(tmp_x2, preproc_french_sentences, batch_size=128, epochs=6, validation_split=.2)

Train on 110288 samples, validate on 27573 samples
Epoch 1/6
110288/110288 [==============================] - 47s 426us/step - loss: 1.0907 - acc: 0.7446 - val_loss: nan - val_acc: 0.8825
Epoch 2/6
110288/110288 [==============================] - 46s 420us/step - loss: 0.2991 - acc: 0.9019 - val_loss: nan - val_acc: 0.9151
Epoch 3/6
110288/110288 [==============================] - 46s 421us/step - loss: 0.2300 - acc: 0.9213 - val_loss: nan - val_acc: 0.9233
Epoch 4/6
110288/110288 [==============================] - 46s 420us/step - loss: 0.2033 - acc: 0.9288 - val_loss: nan - val_acc: 0.9287
Epoch 5/6
110288/110288 [==============================] - 46s 421us/step - loss: 0.1895 - acc: 0.9325 - val_loss: nan - val_acc: 0.9334
Epoch 6/6
110288/110288 [==============================] - 46s 420us/step - loss: 0.1815 - acc: 0.9348 - val_loss: nan - val_acc: 0.9337


In [27]:
embed_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 21, 128)           25472     
_________________________________________________________________
gru_4 (GRU)                  (None, 21, 512)           984576    
_________________________________________________________________
time_distributed_4 (TimeDist (None, 21, 344)           176472    
_________________________________________________________________
activation_4 (Activation)    (None, 21, 344)           0         
Total params: 1,186,520
Trainable params: 1,186,520
Non-trainable params: 0
_________________________________________________________________


In [28]:
np.shape(tmp_x[:1])

(1, 21)

In [29]:
np.shape(embed_model.predict(tmp_x2[:1]))

(1, 21, 344)

In [30]:
np.shape(embed_model.predict(tmp_x2[:1])[0])

(21, 344)

In [31]:
tokenizer= french_tokenizer
index_to_words = {id: word for word, id in tokenizer.word_index.items()}

In [32]:
index_to_words

{1: 'est',
 2: 'en',
 3: 'il',
 4: 'les',
 5: 'mais',
 6: 'et',
 7: 'la',
 8: 'parfois',
 9: 'jamais',
 10: 'le',
 11: "l'",
 12: 'généralement',
 13: 'moins',
 14: 'aimé',
 15: 'au',
 16: 'fruit',
 17: 'préféré',
 18: 'agréable',
 19: 'froid',
 20: 'son',
 21: 'chaud',
 22: 'de',
 23: 'plus',
 24: 'automne',
 25: 'mois',
 26: 'à',
 27: 'elle',
 28: 'citrons',
 29: 'paris',
 30: 'inde',
 31: 'unis',
 32: 'états',
 33: 'france',
 34: 'jersey',
 35: 'new',
 36: 'chine',
 37: 'pendant',
 38: 'pamplemousse',
 39: 'mon',
 40: 'votre',
 41: 'juin',
 42: 'printemps',
 43: 'janvier',
 44: 'hiver',
 45: 'mars',
 46: 'été',
 47: 'mai',
 48: 'septembre',
 49: 'juillet',
 50: 'avril',
 51: 'novembre',
 52: 'décembre',
 53: 'février',
 54: 'octobre',
 55: 'aime',
 56: 'août',
 57: 'merveilleux',
 58: 'relaxant',
 59: 'doux',
 60: 'humide',
 61: 'notre',
 62: 'californie',
 63: 'sec',
 64: 'leur',
 65: 'occupé',
 66: 'pluvieux',
 67: 'calme',
 68: 'beau',
 69: 'habituellement',
 70: 'pommes',
 71: '

In [33]:
logits= embed_model.predict(tmp_x2[:1])[0]

In [34]:
logits

array([[  6.13671943e-07,   4.96470193e-07,   1.75020631e-07, ...,
          1.79742283e-08,   2.51990642e-08,   2.65987161e-08],
       [  1.02520607e-06,   4.15773866e-06,   5.21383697e-07, ...,
          3.40982138e-11,   1.58413102e-10,   3.88263727e-10],
       [  5.79649466e-08,   7.53442883e-01,   6.37759214e-08, ...,
          5.36829123e-11,   5.29490098e-11,   1.88870156e-10],
       ..., 
       [  7.61593938e-01,   4.84160489e-09,   4.42084854e-07, ...,
          1.44549239e-14,   9.66014310e-15,   4.09258097e-14],
       [  7.61593997e-01,   1.15671139e-09,   2.73895182e-08, ...,
          2.63032123e-14,   2.41791145e-14,   6.10125625e-14],
       [  7.61594117e-01,   6.55337784e-10,   1.01377751e-07, ...,
          1.06608371e-14,   5.30768124e-15,   1.83588259e-14]], dtype=float32)

In [35]:
np.argmax(logits, axis=1)

array([ 35,  34,   1,   8,  67,   2,  11,  24,   6,   3,   1, 112,   2,
        50,   0,   0,   0,   0,   0,   0,   0])

In [36]:
logits_to_text(embed_model.predict(tmp_x2[:1])[0], french_tokenizer)

"new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>"

In [37]:
# TODO: Print prediction(s)

# print(logits_to_text(embed_model.predict(tmp_x[:1][0], french_tokenizer))
# # Print prediction(s)
print(logits_to_text(embed_model.predict(tmp_x2[:1])[0], french_tokenizer))


new jersey est parfois calme en l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [38]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    
    learning_rate= 1e-3
    input_seq= Input(input_shape[1:])
    rnn = Bidirectional(GRU(512, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    model = Model(input_seq, Activation('tanh')(logits))
    model.compile(loss= sparse_categorical_crossentropy,
             optimizer=Adam(learning_rate),
             metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)



# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
print(tmp_x.shape)
tmp_x2 = tmp_x.reshape(-1, preproc_french_sentences.shape[-2], 1)
print(preproc_french_sentences.shape[-2])
print(tmp_x2.shape)
# TODO: Train the neural network

bd_model = bd_model(tmp_x2.shape, 
                          max_french_sequence_length,
                          english_vocab_size, french_vocab_size)
bd_model.fit(tmp_x2, preproc_french_sentences, batch_size=128, epochs=10, validation_split=.2)

(137861, 21)
21
(137861, 21, 1)
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 72s 653us/step - loss: 1.2014 - acc: 0.6547 - val_loss: nan - val_acc: 0.7026
Epoch 2/10
110288/110288 [==============================] - 71s 647us/step - loss: 0.8261 - acc: 0.7175 - val_loss: nan - val_acc: 0.7414
Epoch 3/10
110288/110288 [==============================] - 71s 647us/step - loss: 0.6561 - acc: 0.7752 - val_loss: nan - val_acc: 0.8048
Epoch 4/10
110288/110288 [==============================] - 71s 647us/step - loss: 0.5357 - acc: 0.8216 - val_loss: nan - val_acc: 0.8419
Epoch 5/10
110288/110288 [==============================] - 71s 647us/step - loss: 0.4898 - acc: 0.8333 - val_loss: nan - val_acc: 0.8444
Epoch 6/10
110288/110288 [==============================] - 71s 646us/step - loss: 0.4279 - acc: 0.8567 - val_loss: nan - val_acc: 0.8686
Epoch 7/10
110288/110288 [==============================] - 71s 646us/step - loss: 0.3936

In [39]:
preproc_english_sentences.shape

(137861, 15)

In [40]:
preproc_french_sentences.shape

(137861, 21, 1)

In [41]:
bd_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 21, 1024)          1579008   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 21, 344)           352600    
_________________________________________________________________
activation_6 (Activation)    (None, 21, 344)           0         
Total params: 1,931,608
Trainable params: 1,931,608
Non-trainable params: 0
_________________________________________________________________


In [42]:
preproc_english_sentences.shape

(137861, 15)

### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [43]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    lr = 1e-3
    input_seq = Input(input_shape[1:])
    print(input_shape[1:], 'input shape')
    print(tmp_x.shape ,'x_shape')
    gru= GRU(512, return_sequences=True)(input_seq)
    gru2= GRU(256)(gru)
    reptr = RepeatVector(21)(gru2)
    
    decoder = GRU(512, return_sequences=True)(reptr)
    print(reptr.shape, 'reptr.shape')
    print(decoder.shape, 'decoder.shape')

    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
    model = Model(inputs= input_seq, outputs= logits)
    model.compile(loss = sparse_categorical_crossentropy,
                 optimizer=Adam(lr),
                 metrics=['accuracy'])


    
    return model
tests.test_encdec_model(encdec_model)

    
    
# # TODO: Train and Print prediction(s)

tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_english_sequence_length,
    english_vocab_size,
    french_vocab_size)
encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)

print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

(15, 1) input shape
(137861, 21) x_shape
(?, 21, 256) reptr.shape
(?, ?, 512) decoder.shape
(15, 1) input shape
(137861, 15, 1) x_shape
(?, 21, 256) reptr.shape
(?, ?, 512) decoder.shape
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 94s 848us/step - loss: 1.5311 - acc: 0.6004 - val_loss: nan - val_acc: 0.6524
Epoch 2/10
110288/110288 [==============================] - 93s 840us/step - loss: 1.1387 - acc: 0.6660 - val_loss: nan - val_acc: 0.6882
Epoch 3/10
110288/110288 [==============================] - 93s 840us/step - loss: 1.0215 - acc: 0.6838 - val_loss: nan - val_acc: 0.6913
Epoch 4/10
110288/110288 [==============================] - 93s 840us/step - loss: 0.8884 - acc: 0.7199 - val_loss: nan - val_acc: 0.7385
Epoch 5/10
110288/110288 [==============================] - 93s 842us/step - loss: 0.7380 - acc: 0.7545 - val_loss: nan - val_acc: 0.7804
Epoch 6/10
110288/110288 [==============================] - 93s 840us/st

In [44]:
encdec_rnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 15, 1)             0         
_________________________________________________________________
gru_10 (GRU)                 (None, 15, 512)           789504    
_________________________________________________________________
gru_11 (GRU)                 (None, 256)               590592    
_________________________________________________________________
repeat_vector_2 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_12 (GRU)                 (None, 21, 512)           1181184   
_________________________________________________________________
time_distributed_8 (TimeDist (None, 21, 344)           176472    
Total params: 2,737,752
Trainable params: 2,737,752
Non-trainable params: 0
_________________________________________________________________


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [45]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    lr = 1e-3
    input_seq = Input(input_shape[1:])
    print(input_seq.shape, 'input_seq')
    embedding = Embedding(input_dim = english_vocab_size, output_dim=256)(input_seq)
    print(embedding.shape, 'embedding_seq')
    print(tmp_x.shape, 'tmp_x')
    gru = Bidirectional(GRU(512, return_sequences=True))(embedding)
    gru2 = GRU(256)(gru)
    repeatr = RepeatVector(21)(gru2)
    print(repeatr.shape, 'repeatr_')
    decoder = GRU(512, return_sequences=True)(repeatr)
    logits= TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)
    print(logits.shape, 'logits_')
    model = Model(inputs= input_seq, outputs=logits)
    model.compile(loss = sparse_categorical_crossentropy,
                  metrics=['accuracy'],
                  optimizer=Adam(lr)
                 )
    
    return model
tests.test_model_final(model_final)



(?, 15) input_seq
(?, 15, 256) embedding_seq
(137861, 15, 1) tmp_x
(?, 21, 256) repeatr_
(?, 21, 344) logits_


In [46]:

print('Final Model Loaded')
# TODO: Train the final model

tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
tmp_x = tmp_x.reshape(*tmp_x.shape)
print(tmp_x.shape)
model_final_r = model_final(
                            tmp_x.shape,
                            max_english_sequence_length,
                            english_vocab_size,
                            french_vocab_size
                            )

model_final_r.fit(tmp_x, preproc_french_sentences, batch_size=128, epochs=10, validation_split=0.2)
print(logits_to_text(model_final_r.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded
(137861, 15)
(?, 15) input_seq
(?, 15, 256) embedding_seq
(137861, 15) tmp_x
(?, 21, 256) repeatr_
(?, 21, 344) logits_
Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 135s 1ms/step - loss: 1.4762 - acc: 0.6254 - val_loss: nan - val_acc: 0.7398
Epoch 2/10
110288/110288 [==============================] - 133s 1ms/step - loss: 0.6441 - acc: 0.8043 - val_loss: nan - val_acc: 0.8638
Epoch 3/10
110288/110288 [==============================] - 133s 1ms/step - loss: 0.2896 - acc: 0.9148 - val_loss: nan - val_acc: 0.9394
Epoch 4/10
110288/110288 [==============================] - 133s 1ms/step - loss: 0.1631 - acc: 0.9516 - val_loss: nan - val_acc: 0.9587
Epoch 5/10
110288/110288 [==============================] - 133s 1ms/step - loss: 0.1223 - acc: 0.9628 - val_loss: nan - val_acc: 0.9645
Epoch 6/10
110288/110288 [==============================] - 133s 1ms/step - loss: 0.0997 - acc: 0.9695 - val_loss: nan - val_

In [47]:
model_final_r.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 256)           50944     
_________________________________________________________________
bidirectional_4 (Bidirection (None, 15, 1024)          2362368   
_________________________________________________________________
gru_17 (GRU)                 (None, 256)               983808    
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 256)           0         
_________________________________________________________________
gru_18 (GRU)                 (None, 21, 512)           1181184   
_________________________________________________________________
time_distributed_10 (TimeDis (None, 21, 344)           176472    
Total para

## Prediction (IMPLEMENTATION)

In [49]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    print(x.shape, y.shape)
    model = model_final(
        x.shape, y.shape,
        english_vocab_size,
        french_vocab_size
    )
    
    model.fit(x, y, batch_size=128, validation_split=.2, epochs = 6)
    
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

(137861, 15) (137861, 21, 1)
(?, 15) input_seq
(?, 15, 256) embedding_seq
(137861, 15) tmp_x
(?, 21, 256) repeatr_
(?, 21, 344) logits_
Train on 110288 samples, validate on 27573 samples
Epoch 1/6
110288/110288 [==============================] - 135s 1ms/step - loss: 1.4142 - acc: 0.6365 - val_loss: nan - val_acc: 0.7567
Epoch 2/6
110288/110288 [==============================] - 133s 1ms/step - loss: 0.5939 - acc: 0.8183 - val_loss: nan - val_acc: 0.8728
Epoch 3/6
110288/110288 [==============================] - 133s 1ms/step - loss: 0.2863 - acc: 0.9137 - val_loss: nan - val_acc: 0.9436
Epoch 4/6
110288/110288 [==============================] - 133s 1ms/step - loss: 0.1616 - acc: 0.9518 - val_loss: nan - val_acc: 0.9562
Epoch 5/6
110288/110288 [==============================] - 133s 1ms/step - loss: 0.1145 - acc: 0.9653 - val_loss: nan - val_acc: 0.9688
Epoch 6/6
110288/110288 [==============================] - 132s 1ms/step - loss: 0.0993 - acc: 0.9696 - val_loss: nan - val_acc: 0.96

## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [51]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 411149 bytes to machine_translation.html',
 '[NbConvertApp] Converting notebook machine_translation-zh.ipynb to html',
 '[NbConvertApp] Writing 328544 bytes to machine_translation-zh.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?